In [ ]:
#libraries
import requests
from io import BytesIO

import numpy as np
import time
import os
import matplotlib.pyplot as plt
from PIL import Image
import pickle

In [ ]:
def get_latest_png_object():
    objects = client.list_objects(bucket_name, prefix=prefix, recursive=True)
    png_objects = [obj for obj in objects if obj.object_name.endswith('.png')]
    if not png_objects:
        return None
    png_objects.sort(key=lambda x: x.last_modified, reverse=True)
    return png_objects[0]
def show_image(obj_key,output):
    response = client.get_object(bucket_name, obj_key)
    image_data = response.read()

    # Save png file in output/
    os.makedirs(output, exist_ok=True)
    local_path = os.path.join(output, os.path.basename(obj_key))
    with open(local_path, "wb") as f:
        f.write(image_data)
    print(f"Image saved inº : {local_path}")

    # Show image
    image = Image.open(BytesIO(image_data))
    plt.figure()
    plt.imshow(image)
    plt.axis('off')
    plt.title(os.path.basename(obj_key))
    plt.show()

In [ ]:
minio_credencials= os.getenv("MINIO_CREDENCIALS","minio_token1.pkl")

if not minio_credencials.endswith(".pkl"):
    minio_credencials += ".pkl"
# Load configuration from .pkl file
with open(minio_credencials, "rb") as f:
    config = pickle.load(f)
# Create MinIO client
client = Minio(
    endpoint=config["endpoint"],
    access_key=config["access_key"],
    secret_key=config["secret_key"],
    secure= True
)
print("MinIO client loaded from configuration.")

In [ ]:
#Necessary environment variables

bucket_name= os.getenv("BUCKET_NAME","")
out= os.getenv("PREFIX","")
prefix=out+"/"
output= os.getenv("OUTPUT","output")
download_dir = output  
os.makedirs(download_dir, exist_ok=True)

In [ ]:
# Inicial State
print(f"Waiting for new .png file in {bucket_name}/{prefix}...")
initial_obj = get_latest_png_object()
last_seen_key = initial_obj.object_name if initial_obj else None
last_seen_time = initial_obj.last_modified if initial_obj else None

while True:
    latest_obj = get_latest_png_object()
    if latest_obj:
        new_key = latest_obj.object_name
        new_time = latest_obj.last_modified
        if new_key != last_seen_key or new_time != last_seen_time:
            print(f"File detected: {new_key}")
            local_png_path = os.path.join(download_dir, os.path.basename(new_key))
            client.fget_object(bucket_name, new_key, local_png_path)
            print(f"Saved in: {local_png_path}")
            show_image(new_key, output)  
            break 
    time.sleep(5)